In [ ]:
# -*- coding: utf-8 -*-
# USAGE
# python detect_mask_image.py --image examples/example_01.png

# importer les bibliothèque nécessaires
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os


#diviser le chemin en 2
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join(["face_detector", "deploy.prototxt"])#le chemin vers le modèle de détecteur de visage
weightsPath = os.path.sep.join(["face_detector",
	"res10_300x300_ssd_iter_140000.caffemodel"])#le chemin vers les poids du modèle de détecteur de visage
net = cv2.dnn.readNet(prototxtPath, weightsPath)#lire le modèle avec son poid

# charger le model
print("[INFO] loading face mask detector model...")
model = load_model("mask_detector.model")#charger le modèle de détecteur de masque construit précedemment

# chargement de l'image
image = cv2.imread("examples/example_01.png")
orig = image.copy()#faire une copie de l'image
(h, w) = image.shape[:2]#hauteur et largeur de l'image#h=400 et w=600

# prétraitememnt de l'image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
	(104.0, 177.0, 123.0))
#image: Il s'agit de l'image d'entrée que nous voulons prétraiter avant de la transmettre à notre réseau neuronal profond pour la classification.
#scalefactor: Après avoir effectué une soustraction moyenne, nous pouvons éventuellement mettre à l'échelle nos images par un certain facteur. Cette valeur par défaut est «1.0» (c'est-à-dire pas de mise à l'échelle) mais nous pouvons également fournir une autre valeur. Il est également important de noter que scalefactor doit être 1 / \ sigma car nous multiplions en fait les canaux d'entrée (après soustraction de la moyenne) par scalefactor.
#size: Ici, nous fournissons la taille spatiale attendue par le réseau de neurones convolutifs. Pour la plupart des réseaux de neurones de pointe actuels, il s'agit de 224 × 224, 227 × 227 ou 299 × 299.
#signifie: ce sont nos valeurs moyennes de soustraction. Ils peuvent être un 3-tuple des moyens RVB ou ils peuvent être une valeur unique, auquel cas la valeur fournie est soustraite de chaque canal de l'image. Si vous effectuez une soustraction moyenne, assurez-vous de fournir le 3-tuple dans l'ordre `(R, V, B)`, en particulier lorsque vous utilisez le comportement par défaut de swapRB = True.
#swapRB: OpenCV suppose que les images sont dans l'ordre des canaux BGR; cependant, la valeur «moyenne» suppose que nous utilisons l'ordre RVB. Pour résoudre cet écart, nous pouvons permuter les canaux R et B dans l'image en définissant cette valeur sur «True». Par défaut, OpenCV effectue ce changement de chaîne pour nous.
#La fonction cv2.dnn.blobFromImage renvoie un blob qui est notre image d'entrée après la soustraction moyenne, la normalisation et le changement de canal.
# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()
#probabilité que la détection soit bien un visage
# les détections
nbr_vis=0
avec=0
sans=0
faces=[]
for i in range(0, detections.shape[2]):
	confidence = detections[0, 0, i, 2]#la probabilité que ca soit bien un visage

	#filtrer les détections faibles qui seront considérés comme des erreurs


	if confidence > 0.4:
		#calculer les coordonnées des limites de la régions d'intérêt
		nbr_vis+=1         
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

		(startX, startY, endX, endY) = box.astype("int")

		#la ROI(Region of interest) doit être dans l'image
		(startX, startY) = (max(0, startX), max(0, startY))
		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

		# extraire la ROI, convertir le canal du BGR au RGB ordre 
		# redimensionnement et le prétraitement du visage
		face = image[startY:endY, startX:endX]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
		face = cv2.resize(face, (224, 224))
		face = img_to_array(face)
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)
		faces.append(face)
		# prédire avec le modèle si le visage est aec ou sans mask
		(mask, withoutMask) = model.predict(face)[0]


		# déterminer la plus grande probabilité et ajouter du texte et un rectangle au visage détecté
		if mask > withoutMask:
		    label="Mask"
		    avec+=1
		else: 
		    label="No mask"
		    sans+=1
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(image, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

#l'image de sortie
from PIL import Image
import imageio
if nbr_vis==1:
    label1="Il y'a {} personne dont {} avec masque et {} sans masque ".format(nbr_vis,avec,sans)
elif nbr_vis>=2:
    label1="Il y'a {} personnes dont {} avec masque et {} sans masque ".format(nbr_vis,avec,sans)
else:
    label1="Il n'y a personne"
    
cv2.putText(image, label1 , (50,40),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255,255), 2)
cv2.imshow("Output", image)
im1=Image.fromarray(image)
imageio.imsave("Output.png", im1)


cv2.waitKey(0)



[INFO] loading face detector model...
[INFO] loading face mask detector model...
[INFO] computing face detections...
